In [2]:
import boto3
s3=boto3.resource('s3')

In [6]:
bucket_name='fractionatordata'
try:
    s3.create_bucket(Bucket=bucket_name)
    print("S3 bucket {} was created".format(bucket_name))
except Exception as e:
    print("S3 error: ",e)

S3 bucket fractionatordata was created


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from scipy import stats
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from datetime import date
from sklearn.preprocessing import MinMaxScaler


In [9]:
df=pd.read_csv('Fractionator.csv')

In [10]:
df['Time']=pd.to_datetime(df['Time'])

In [11]:
df=df[(np.abs(stats.zscore(df["AI_2020_OVERHEAD_C5'S_MOL"])) < 3)]

In [12]:
df.rename(columns = {"AI_2020_OVERHEAD_C5'S_MOL":"AI_top","AI_2021_MIDDLE_C7'S_MOL":"AI_middle","AI_2022_BOTTOM_C3'S_MOL":"AI_bottom",
                          'FIC_2100_PV_FEED_FURNACE_FUEL_SCFH':'Furnace' ,'FIC_2004_PV_TOP_REFLUX_MBBL/D':'Reflux_top',
                    'FI-2005_PV_FEED_FLOW_MBBL/D':'Feed'
 }, inplace = True)

In [13]:
df_uni=df[['AI_top','Furnace','Feed','Reflux_top',]]

In [14]:
df_Xgb=df_uni[['AI_top']]

In [15]:
df_Xgb.head()

,AI_top
0,2.00000
1,2.00360
2,2.00870
3,2.00385
4,1.99852


In [64]:
def df_to_X_y(df, window_size=10):
  df_np = df_Xgb.to_numpy()
  X = []
  y = []
  for i in range(len(df_np)-window_size):
    row = [r for r in df_np[i:i+window_size]]
    X.append(row)
    label = df_np[i+window_size][0]
    y.append(label)
  return np.array(X), np.array(y)

In [65]:
X,y= df_to_X_y(df_Xgb,window_size=10)
X.shape, y.shape

((5719, 10, 1), (5719,))

In [66]:
y=y.reshape([5719])
print(y.shape)
X=X.reshape([5719,10])
X.shape

(5719,)


(5719, 10)

In [69]:
X_dataframe=pd.DataFrame(X,columns=['t0','t1','t2','t3','t4','t5','t6','t7','t8','t9'])
X_dataframe.head()

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9
0,2.00000,2.00360,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368
1,2.00360,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608
2,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237
3,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237,2.02020
4,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237,2.02020,2.01924


In [70]:
y_dataframe=pd.DataFrame(y,columns=['t10'])
y_dataframe.head()

,t10
0,2.01608
1,2.02237
2,2.02020
3,2.01924
4,2.01335


In [96]:
xy=pd.DataFrame()
xy['t10']=y_dataframe['t10']
xy[['t0','t1','t2','t3','t4','t5','t6','t7','t8','t9']]=X_dataframe[['t0','t1','t2','t3','t4','t5','t6','t7','t8','t9']]
xy.head()



,t10,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9
0,2.01608,2.00000,2.00360,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368
1,2.02237,2.00360,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608
2,2.02020,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237
3,2.01924,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237,2.02020
4,2.01335,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237,2.02020,2.01924


In [159]:
xy.tail(5)

,t10,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9
5714,1.66446,1.71108,1.71109,1.69975,1.69506,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325
5715,1.66227,1.71109,1.69975,1.69506,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325,1.66446
5716,1.65845,1.69975,1.69506,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325,1.66446,1.66227
5717,1.67002,1.69506,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325,1.66446,1.66227,1.65845
5718,1.67681,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325,1.66446,1.66227,1.65845,1.67002


In [94]:
df_data_features=xy.iloc[:,1:]
df_data_features

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9
0,2.00000,2.00360,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368
1,2.00360,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608
2,2.00870,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237
3,2.00385,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237,2.02020
4,1.99852,1.99710,2.00260,2.01440,2.01669,2.02368,2.01608,2.02237,2.02020,2.01924
...,...,...,...,...,...,...,...,...,...,...
5714,1.71108,1.71109,1.69975,1.69506,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325
5715,1.71109,1.69975,1.69506,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325,1.66446
5716,1.69975,1.69506,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325,1.66446,1.66227
5717,1.69506,1.68498,1.68894,1.68242,1.68244,1.67338,1.67325,1.66446,1.66227,1.65845


In [160]:
df_data_target=xy.iloc[:,0].rename("Targets")
df_data_target

0       2.01608
1       2.02237
2       2.02020
3       2.01924
4       2.01335
         ...   
5714    1.66446
5715    1.66227
5716    1.65845
5717    1.67002
5718    1.67681
Name: Targets, Length: 5719, dtype: float64

In [97]:
split_index = int(len(xy) * 0.8)

split_index

4575

In [166]:
train_data,test_data=np.split(xy,[int(len(xy)*0.8)] )
train_data.shape,test_data.shape

((4575, 11), (1144, 11))

In [138]:
train_data=xy.iloc[:split_index,:]
test_data=xy.iloc[split_index:,:]
#,test_data=np.split(xy,split_index)
print(train_data.shape,test_data.shape)

(4575, 11) (1144, 11)


In [150]:

X_train = X[:split_index]
X_test = X[split_index:]

y_train = y[:split_index]
y_test = y[split_index:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4575, 10), (1144, 10), (4575,), (1144,))

In [169]:
import os
prefix="xgboost_ai_top"
train_csv_path='s3://{}/{}/{}/{}'.format(bucket_name,prefix,'train','train.csv')
test_csv_path='s3://{}/{}/{}/{}'.format(bucket_name,prefix,'test','test.csv')
print(train_csv_path)
print(test_csv_path)


s3://fractionatordata/xgboost_ai_top/train/train.csv
s3://fractionatordata/xgboost_ai_top/test/test.csv


In [170]:
train_data.to_csv(train_csv_path,index=False,header=False)
test_data.to_csv(test_csv_path,index=False,header=False)


In [171]:
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

In [172]:
xgboost_container=image_uris.retrieve('xgboost',boto3.Session().region_name,"1.5-1")
display(xgboost_container)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1'

In [146]:
! pip install xgboost
 

  Using cached xgboost-1.6.2-py3-none-manylinux2014_x86_64.whl (255.9 MB)


In [148]:
import xgboost as xgb

reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=1000,
                       early_stopping_rounds=50,
                       objective='reg:linear',
                       max_depth=3,
                       learning_rate=0.01)

In [118]:
hyperparameters={"base_score":0.5, 
                 "booster":'gbtree',    
                 "n_estimators":1000,
                 "early_stopping_rounds":50,
                 "objective":'reg:linear',
                 "max_depth":"3",
                 "learning_rate":"0.01"
                }

In [173]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:linear",
        "num_round":"50"}


In [112]:
output_path="s3://{}/{}/{}/".format(bucket_name,prefix,'output')
print(output_path)

s3://fractionatordata/xgboost_ai_top/output/


In [174]:
estimator=sagemaker.estimator.Estimator(image_uri=xgboost_container,
                                        hyperparameters=hyperparameters,
                                        role=sagemaker.get_execution_role(),
                                        instance_count=1,
                                        instance_type='ml.m4.xlarge',
                                        volume_size=5,
                                        #use_spot_instances=True,
                                       # max_run=300,
                                        #max_wait=600
                                       )

In [175]:
content_type="csv"
train_input=TrainingInput("s3://{}/{}/{}/".format(bucket_name,prefix,'train',content_type=content_type))
test_input=TrainingInput("s3://{}/{}/{}/".format(bucket_name,prefix,'test',content_type=content_type))

In [176]:
estimator.fit({'train':train_input,'validation':test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-04-22-20-51-14-256


2023-04-22 20:51:14 Starting - Starting the training job...
2023-04-22 20:51:40 Starting - Preparing the instances for training......
2023-04-22 20:52:55 Downloading - Downloading input data......
2023-04-22 20:53:31 Training - Downloading the training image...
2023-04-22 20:54:06 Training - Training image download completed. Training in progress..[2023-04-22 20:54:22.964 ip-10-2-119-83.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-04-22 20:54:23.044 ip-10-2-119-83.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-04-22:20:54:23:INFO] Imported framework sagemaker_xgboost_container.training
[2023-04-22:20:54:23:INFO] Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
[2023-04-22:20:54:23:INFO] No GPUs detected (normal if no gpus installed)
[2023-04-22:20:54:23:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-04-22:20:54:23:INFO] Determined 0 GPU(s) available on the instance.

UnexpectedStatusException: Error for Training job sagemaker-xgboost-2023-04-22-20-51-14-256: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/training.py", line 102, in main
    train(framework.training_env())
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/training.py", line 98, in train
    run_algorithm_mode()
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/training.py", line 64, in run_algorithm_mode
    sagemaker_train(
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/algorithm_mode/train.py", line 198, in sagemaker_train
    train_dmatrix, val_dmatrix, train_val_dmatrix = get_validated_dmatrices(
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/a

In [ ]:
from sagemaker.serializers import CSVSerlializer
xgb_predictor=estimaor.deploy(instance_cout=1,instance_type='ml.m4.xlarge',serializer=CSVSerlializer)

In [ ]:
endpoint_AI_top=xgb_predictor.endpoint_name

In [185]:
hyperparams = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "reg:squarederror",
    "num_round": "50",
    "verbosity": "2",
}

instance_type = "ml.m5.2xlarge"
output_path =output_path
content_type = "csv"

In [188]:
# Open Source distributed script mode
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

session = Session()
script_path = "train.py"

xgb_script_mode_estimator = XGBoost(
    entry_point=script_path,
    framework_version="1.7-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type=instance_type,
    output_path=output_path,
)


INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.2xlarge.


In [ ]:
xgb_script_mode_estimator.fit({"train": train_input, "validation": test_input})


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-04-22-22-52-31-967


2023-04-22 22:52:32 Starting - Starting the training job...
2023-04-22 22:52:47 Starting - Preparing the instances for training...
2023-04-22 22:53:32 Downloading - Downloading input data...
2023-04-22 22:53:47 Training - Downloading the training image...
2023-04-22 22:54:43 Uploading - Uploading generated training model.[2023-04-22 22:54:36.486 ip-10-2-219-84.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-04-22 22:54:36.545 ip-10-2-219-84.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-04-22:22:54:36:INFO] Imported framework sagemaker_xgboost_container.training
[2023-04-22:22:54:36:INFO] No GPUs detected (normal if no gpus installed)
[2023-04-22:22:54:36:INFO] Invoking user training script.
[2023-04-22:22:54:37:INFO] Module train does not provide a setup.py. 
Generating setup.py
[2023-04-22:22:54:37:INFO] Generating setup.cfg
[2023-04-22:22:54:37:INFO] Generating MANIFEST.in
[2023-04-22:22:54:37:INFO] Installing module w